# 21. Декораторы 1
В первом задании курса предлагается решить классическую задачу с собеседований по питону - написать декоратор, который вычисляет время выполнения оборачиваемой функции. Итак, требования к декоратору:

- декоратор должен называться time_decorator
- он должен вычислять время в секундах, в течение которого выполняется обернутая функция при ее вызове. Количество секунд должно быть выведено сразу после выполнения оборачиваемой функции. Количество секунд следует округлять до целого числа и выводить целое число
- после оборачивания функция должна возвращать тот же результат, который возвращала исходная функция
- пробрасывать аргументы из декоратора в функцию необязательно для выполнения этого задания

#### Пример использования декоратора
```@time_decorator
def sleep_1_sec():
    time.sleep(1)
    print("function")
    return 25```

result = sleep_1_sec()
>function
> 
>1

print(result)
>25

In [1]:
import timeit
def time_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = timeit.default_timer() 
        res = func(*args, **kwargs) 
        print(round(timeit.default_timer() - start_time))
        return res    
    return wrapper

In [2]:
import time
@time_decorator
def sleep_1_sec():
    time.sleep(1)
    print("function")
    return 25

In [3]:
result = sleep_1_sec()

function
1


In [4]:
print(result)

25


# 22. Декораторы 2
В этом задании требуется написать фабрику декораторов, которые будут логировать вызовы функций. Чтобы не усложнять, в качестве журнала событий будем использовать списки. Требования к фабрике:

- фабрика должна называться `logging_decorator`
- при создании декоратора фабрика должна принять требуемый список-логгер в аргументах
- обернутая функция должна возвращать тот же результат, который бы вернула оборачиваемая функция
- при вызове обернутой функции в список-логгер должен добавляться словарь, в котором будут храниться название функции, список поданных аргументов, время вызова функции и результат, который она вернула. Формат словаря должен быть таким:


    {
        'name': 'test_function',
        'arguments': {'a': 1, 'b': 2},
        'call_time': datetime.datetime(2021, 8, 1, 18, 18, 7, 849184),
        'result': 127
    }

Ниже приведен пример использования такого декоратора.

#### Пример использования декоратора

    logger = []  # этот словарь будет хранить наш "лог"

    @logging_decorator(logger)  # в аргументы фабрики декораторов подается логгер
    def test_simple(a, b=2):
        return 127

    test_simple(1)  # при вызове функции в список logger должен добавиться словарь с
                    # информацией о вызове функции

    print(logger)

> `[{'name': 'test_simple', 'arguments': {'a': 1, 'b': 2}, 'call_time': datetime.datetime(2021, 8, 1, 18, 18, 7, 849184), 'result': 127}]`

In [5]:
import datetime
import inspect
logger = []
def logging_decorator(logger):
    def log_dec(func):
        def wrapper(*args, **kwargs):
            Now = datetime.datetime.now()
            name = '{}'.format(func.__name__)
            arguments = inspect.getcallargs(func, *args, **kwargs)
            res = func(*args, **kwargs)
            log_dict = {'name' : name,
                        'arguments' : arguments,
                        'call_time' : Now,
                        'result' : res}
            logger.append(log_dict)
            return res
        return wrapper
    return log_dec  

In [6]:
@logging_decorator(logger)  # в аргументы фабрики декораторов подается логгер
def test_simple(a, b=2):
    return 127

test_simple(1, 2)  # при вызове функции в список logger должен добавиться словарь с
                # информацией о вызове функции

print(logger)

[{'name': 'test_simple', 'arguments': {'a': 1, 'b': 2}, 'call_time': datetime.datetime(2022, 11, 26, 23, 11, 17, 533645), 'result': 127}]


In [7]:
test_simple(2, b = 8)
print(logger)

[{'name': 'test_simple', 'arguments': {'a': 1, 'b': 2}, 'call_time': datetime.datetime(2022, 11, 26, 23, 11, 17, 533645), 'result': 127}, {'name': 'test_simple', 'arguments': {'a': 2, 'b': 8}, 'call_time': datetime.datetime(2022, 11, 26, 23, 11, 17, 625142), 'result': 127}]


# 23. Сеть
[Здесь](https://jsonplaceholder.typicode.com/) описано некоторое API, в котором есть доступ к базе пользователей, постов, комментариев и т.д. Методы, которые мы будем использовать, описаны в разделе Resources. Примеры использования API (правда, на JavaScript'е) описаны на том же сайте по ссылке [Guide](https://jsonplaceholder.typicode.com/guide/). Вам нужно для каждого пользователя посчитать количество оставленных постов и количество оставленных комментариев. Всю информацию для этого нужно стягивать GET-запросами по API. Результат нужно отправить в ваше пространство в https://webhook.site в виде POST-запроса, содержащего JSON следующего формата:

```{
  "statistics": [
    {
      "id": 1,
      "username": "lolkek",
      "email": "user1@mail.dot",
      "posts": 125,
      "comments": 1358
    },
    {
      "id": 2,
      "username": "cheburek",
      "email": "user2@mail.dot",
      "posts": 5,
      "comments": 12
    }
  ]
}```

Поскольку среда исполнения Яндекс-контеста не имеет доступа к интернету, проверить правильность выполнения задания вы можете, отправив в качестве ответа на задание "Сеть" pickle объекта ответа запроса:

```response = requests.post(.....)
with open("solution.pickle", 'wb') as f:
    pickle.dump(response, "solution.py")```
    
И отправляйте тот файл, который появится в результате исполнения этого кода.

In [24]:
import requests
import json

resp_users = requests.get('https://jsonplaceholder.typicode.com/users')

jsonp_users_data = {resp_users['id'] : {
    'id' : resp_users['id'],
    'username' : resp_users['username'],
    'email' : resp_users['email'],
    'posts' : 0,
    'comments' : 0
     } for resp_users in json.loads(resp_users.content)}
users_ids = [id for id in jsonp_users_data]

for id in users_ids:
    resp_posts = requests.get(f'https://jsonplaceholder.typicode.com/users/{id}/posts')
    jsonp_posts_data = json.loads(resp_posts.content)
    jsonp_users_data[id]['posts'] = len(jsonp_posts_data)
    resp_comments = requests.get(f'https://jsonplaceholder.typicode.com/comments')
    jsonp_comments_data = json.loads(resp_comments.content)
    comments_count = 0
    for i in range(len(jsonp_comments_data)):
        if jsonp_users_data[id]['email'] == jsonp_comments_data[i]['email']:
            comments_count += 1
    jsonp_users_data[id]['comments'] = comments_count     
    
response = requests.post('https://webhook.site/ac6e2362-3420-4fef-add2-76e30ec16e4f', 
                        data = json.dumps({"statistics":[jsonp_users_data[id] for id in jsonp_users_data]}))

In [25]:
print(jsonp_users_data)
print()
for id in users_ids:
    print(jsonp_users_data[id]['comments'])
print(response)
print(json.dumps({"statistics":[jsonp_users_data[id] for id in jsonp_users_data]}))

{1: {'id': 1, 'username': 'Bret', 'email': 'Sincere@april.biz', 'posts': 10, 'comments': 0}, 2: {'id': 2, 'username': 'Antonette', 'email': 'Shanna@melissa.tv', 'posts': 10, 'comments': 0}, 3: {'id': 3, 'username': 'Samantha', 'email': 'Nathan@yesenia.net', 'posts': 10, 'comments': 0}, 4: {'id': 4, 'username': 'Karianne', 'email': 'Julianne.OConner@kory.org', 'posts': 10, 'comments': 0}, 5: {'id': 5, 'username': 'Kamren', 'email': 'Lucio_Hettinger@annie.ca', 'posts': 10, 'comments': 0}, 6: {'id': 6, 'username': 'Leopoldo_Corkery', 'email': 'Karley_Dach@jasper.info', 'posts': 10, 'comments': 0}, 7: {'id': 7, 'username': 'Elwyn.Skiles', 'email': 'Telly.Hoeger@billy.biz', 'posts': 10, 'comments': 0}, 8: {'id': 8, 'username': 'Maxime_Nienow', 'email': 'Sherwood@rosamond.me', 'posts': 10, 'comments': 0}, 9: {'id': 9, 'username': 'Delphine', 'email': 'Chaim_McDermott@dana.io', 'posts': 10, 'comments': 0}, 10: {'id': 10, 'username': 'Moriah.Stanton', 'email': 'Rey.Padberg@karina.biz', 'posts'

In [26]:
import pickle
with open("solution.pickle", 'wb') as f:     
    pickle.dump(response, f)